In [2]:
# Place to dowload book free
#https://annas-archive.org/
import spacy
import ebooklib
from ebooklib import epub  
from html.parser import HTMLParser #(usar beutiful soup)
from collections import defaultdict
from spacy.lang.es import Spanish
from bs4 import BeautifulSoup
from lxml import etree
import re
# En spacy, los saltos de linea los ignora, asi que supongo que para separar en parrafos o bien ingnorarlos, o bien procesarlos por separado,
# haciendo un split por \n



In [3]:
# paths and data acquisition 
book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
book = epub.read_epub(book_path)


title = book.get_metadata('DC', 'title')[0][0]
print(f"The book {title} was correctly read")


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\pable\AppData\Local\Temp\ipykernel_11456\82667957.py:2: SyntaxWarning: invalid escape sequence '\A'
  book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


The book Asesinato en el Orient Express was correctly read


c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [4]:
# Function to return all the titles of the book

def get_book_titles(book):
    titles = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_body_content()

            # Step 1: Decode the byte string to a regular string
            decoded_str = content.decode('utf-8')

            # Step 2: Parse the decoded string with BeautifulSoup
            soup = BeautifulSoup(decoded_str, 'html.parser')

            # Step 3: The chapter titles are inside <h2> tag, Find the <h2> tag and get the 'title' attribute
            h2_tag = soup.find('h2')
            if h2_tag and 'title' in h2_tag.attrs:
                title_content = h2_tag['title']
                titles.append(title_content)

    return titles

titles = get_book_titles(book)
print(titles)


['1. El pasajero del «Taurus Express»', '2. El hotel Tokatlian', '3. Poirot renuncia a un caso', '4. Un grito en la noche', '5. El crimen', '6. ¿Una mujer?', '7. El cadáver', '8. El caso Armstrong', '1. Declaración del conductor del coche dormitorio', '2. Declaración del secretario', '3. Declaración del criado', '4. Declaración de la dama norteamericana', '5. Declaración de la dama sueca', '6. Declaración de la princesa rusa', '7. Declaración del conde y la condesa Andrenyi', '8. Declaración del coronel Arbuthnot', '9. Declaración de míster Hardman', '10. Declaración del italiano', '11. Declaración de miss Debenham', '12. Declaración de la doncella alemana', '13. Resumen de las declaraciones de los viajeros', '14. El arma', '15. Los equipajes', '1. ¿Cuál de ellos?', '2. Diez preguntas', '3. Algunos puntos sugestivos', '4. La mancha de grasa en un pasaporte húngaro', '5. El nombre de pila de la princesa Dragomiroff', '6. Una entrevista con el coronel Arbuthnot', '7. La identidad de Mary

In [72]:
# class to store the chapters
class Chapter():
    def __init__(self,chapter_name,chapter_text = None) -> None:
        self.chapter_name = chapter_name
        self.chapter_text = chapter_text

    def set_chapter_text(self,chapter_text):
        self.chapter_text = chapter_text

    def __str__(self) -> str:
        return f"Chapter: {self.chapter_name} \nContent: {self.chapter_text}"
    
    def print_chapter_list(chapter_list):
        for chap in chapter_list:
            print(chap)


In [76]:
# Function to return all the text of the book until a chapter is found via the number of the chapter or the name of the chapter

def get_book_text_until_chapter(book,title_search_method = None):

    chapters = []
    # First if user only knows te title of the chapter, we search for it in all the titles, to ensure it exists and get the index
    title_index = ''

    if isinstance(title_search_method,(int,float)):
        title_index = title_search_method
        chapters = [Chapter(f"Chapter {i}") for i in range(title_index)]

    elif isinstance(title_search_method,str):
        titles = get_book_titles(book)
        for i,title in enumerate(titles):
            if title_search_method in title:
                title_index = i
                print(f"The chapter provided is the chapter {title_index}")
                chapters = [Chapter(f"Chapter {title}") for title in titles[:title_index]]
                break
        
        if title_index is None:
            print(f"Title {title_search_method} not found")
            return None

    else:
        print("No title provided, please provide some title index of text")
        return None

    current_title_index = 0
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_body_content()
            chapter_text = content.decode('utf-8')

            soup = BeautifulSoup(chapter_text, 'html.parser')
            h2_tag = soup.find('h2')

            # Each chapter is saved inside a single ITEM_DOCUMENT
            # If the item is a title and different for the index or text we are looking for, add the text to the book_text var

            if h2_tag and 'title' in h2_tag.attrs:
                if current_title_index == title_index:
                    return chapters
                
                # all the text of the chapter is conformed by the title and a span that indicates some text format info
                # However it ends with </span>, and after that all text is clean text of the chapter with &#13; <p> and </p>&#13; annotations
                # that marks the start and end of a parragraph

                span_class_index = chapter_text.find('<span class="letra">') 
                if span_class_index != -1:
                    chapter_text = chapter_text[span_class_index + len('<span class="letra">'):]
                    chapter_text = re.sub("</span>","",chapter_text) 

                chapter_text = re.sub("(<p>)|(&#13;)|(<\/p>)"," ",chapter_text)    
                chapters[current_title_index].set_chapter_text(chapter_text)

                current_title_index +=1

    return chapters

chapters = get_book_text_until_chapter(book,"Los equipajes")

Chapter.print_chapter_list(chapters)


<>:55: SyntaxWarning: invalid escape sequence '\/'
<>:55: SyntaxWarning: invalid escape sequence '\/'
C:\Users\pable\AppData\Local\Temp\ipykernel_11456\591462692.py:55: SyntaxWarning: invalid escape sequence '\/'
  chapter_text = re.sub("(<p>)|(&#13;)|(<\/p>)"," ",chapter_text)


The chapter provided is the chapter 22
Chapter: Chapter 1. El pasajero del «Taurus Express» 
Content: ERAN las cinco de una madrugada de invierno en Siria. Junto al andén de Alepo estaba detenido el tren que las guías de ferrocarriles designan con el nombre de <i>Taurus Express</i>. Estaba formado por un coche con cocina comedor, un coche cama y dos coches corrientes.  
 
   Junto al estribo del coche cama se encontraba un joven teniente francés, de resplandeciente uniforme, conversando con un hombrecillo embozado hasta las orejas, del que sólo podían verse la punta de la nariz y las dos guías de un enhiesto bigote.  
 
   Hacía un frío intensísimo, y aquella misión de despedir a un distinguido forastero no era cosa de envidiar, pero el teniente Dubosc la cumplía como un valiente. No cesaban de salir de sus labios frases corteses en el más pulido francés. Y no es que estuviese completamente al corriente de los motivos del viaje de aquel personaje. Había habido rumores, naturalmente, co

First we need know which lines are refering to some character or is being talked by them 

PARTE DE TRATAMIENTO

In [7]:
!pip install transformers torch

In [77]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
information = None
character = None
input_text = f"summarize the character of {character} in this passage: {information}"

In [ ]:
#NOTA para separar los chunks en el texto, separar por parrafos si se puede  y 100% por caps sino pues F

In [ ]:
# Function to split input text into manageable chunks (by sentence or paragraph)
def split_text(input_text, max_length=512):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True, padding=True)
    if inputs["input_ids"].shape[1] > max_length:
        # Split the input into paragraphs or sentences here
        paragraphs = input_text.split("\n")
        chunks = []
        current_chunk = []
        current_length = 0
        
        for para in paragraphs:
            tokenized = tokenizer(para, return_tensors="pt")
            token_length = tokenized["input_ids"].shape[1]
            if current_length + token_length > max_length:
                chunks.append(" ".join(current_chunk))
                current_chunk = [para]
                current_length = token_length
            else:
                current_chunk.append(para)
                current_length += token_length
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks
    return [input_text]

# Example of chunking long text
chunks = split_text(input_text)

In [ ]:
# Example function to summarize a chunk of text
def generate_summary(input_text):
    input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)
    output_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

# Get summaries for all chunks
summaries = [generate_summary(chunk) for chunk in chunks]
for summary in summaries:
    print(summary)